Description and information about the project goes here

In [ ]:
# Imports
import pandas as pd
import numpy as np
import openpyxl
import xlsxwriter
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string
import string
import spacy
from emotion import Emotion
import json
from wnaffect import WNAffect
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import heapq

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/protago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/protago/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/protago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/protago/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

Task 1
Add task description

In [8]:
# Extracts all line numbers of lines in the specified topic. The topic_number argument must be given as a string. For example: '9'
# Topic number 9 is politics
def save_topic_lines(path_to_topic_file, topic_number):

    topic_lines = []
    topic_number = str(topic_number)

    with open(path_to_topic_file, 'r') as file:
        
        i = 1

        for line in file:
            if line[0] == topic_number:
                topic_lines.append(i)
            
            i = i + 1

    return topic_lines

# Extracts all dialogue lines from a specific topic
# if topic is 'all', every topic is extracted
def extract_topic(path_to_dialogue_file, path_to_topic_file, topic_number):

    topic_lines = save_topic_lines(path_to_topic_file, topic_number)
    topic_dialogue = []

    with open(path_to_dialogue_file, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file):

            if topic_number == 'all':
                topic_dialogue.append(line)

            elif topic_number != 'all':
                if line_number in topic_lines:
                    topic_dialogue.append(line)

    return topic_dialogue

# Creates a pandas dataframe for the dialogue data in a specific topic
# Rows are dialogue lines. They are in the same order as in the original dialogues_text.txt file
# Columns are utterances in that dialogue.
def create_topic_dataframe(path_to_dialogue_file, path_to_topic_file, topic_number):

    topic_dialogue = extract_topic(path_to_dialogue_file, path_to_topic_file, topic_number)


    #split_dialogue = [line.split('__eou__') for line in topic_dialogue]

    split_dialogue = []

    for line in topic_dialogue:

        split_line = line.split('__eou__')

        for i in range(len(split_line)):
            
            if split_line[i][0] == " ":

                new_line = split_line[i][1:]
                split_line[i] = new_line

            if split_line[i][-1] == " ":

                new_line = split_line[i][:-1]
                split_line[i] = new_line


        split_dialogue.append(split_line)


    topic_dialogue_data = pd.DataFrame(split_dialogue)

    return topic_dialogue_data

# Saves the dataframe in excel format
# This is just for not having to write the annoying file format
def save_dataframe_as_excel(data, filename):

    if '.xlsx' not in filename:
        filename = filename + '.xlsx'

    data.to_excel(filename, header=False, index=False)

# Does everything above. Extracts the topic, makes it into a dataframe and saves in excel format
# if topic number is 'all', every topic is extracted
def extract_and_save_topic_dialogue(path_to_dialogue_file, path_to_topic_file, topic_number, filename):

    topic_dialogue_data = create_topic_dataframe(path_to_dialogue_file, path_to_topic_file, topic_number)
    save_dataframe_as_excel(topic_dialogue_data, filename)

# Give topic number and name of file to save the data. The topic number can be given as a string or an integer.
extract_and_save_topic_dialogue('ijcnlp_dailydialog/dialogues_text.txt', 'ijcnlp_dailydialog/dialogues_topic.txt', 9, 'topic9data')

In [10]:
# Open the data that was made with task1_save_topic.py
def open_process_data(name_of_excel_file):
    
    data = pd.read_excel(name_of_excel_file)
    rows, columns = data.shape
    dialogues = []
    
    for row in range(rows):

        utterances = []

        for column in range(columns):
            if type(data.iat[row, column]) is str:
                utterances.append(data.iat[row, column])
            
        dialogues.append(utterances)

    return dialogues

# Tokenizes, lowers and removes special characters from data
def tokenize_data(dialogues):
    
    special = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '=', '+', '[', ']', '{', '}', ';', ':', '"', "'", '<', '>', ',', '.', '/', '?', '\\', '|', '`', '~', '...']
    tokenized_dialogues = []

    for dialogue in dialogues:
        tokenized_dialogue = []

        for utterance in dialogue:
            processed_tokenized_utterance = []
            tokenized_utterance = word_tokenize(utterance)

            for token in tokenized_utterance:
                token.lower()
                if (token not in special) and (len(token) != 1):
                    processed_tokenized_utterance.append(token)
                
            tokenized_dialogue.append(processed_tokenized_utterance)
        
        tokenized_dialogues.append(tokenized_dialogue)
    
    return tokenized_dialogues

# Calculates the vocabulary size for data. Data is given in the form that tokenize_data returns it
def vocabulary_size(dialogues):
    
    tokenized_dialogues = tokenize_data(dialogues)
    counted_words = []
    vocabulary_size = 0

    for dialogue in tokenized_dialogues:
        for utterance in dialogue:
            for token in utterance:
                if token not in counted_words:
                    vocabulary_size += 1
                    counted_words.append(token)
                    #print("Unique token: " + token)

    return vocabulary_size

# Calculates the number of utterances for a dialogue
def count_utterances(dialogues):

    num_of_utterances = 0

    for dialogue in dialogues:
        for utterance in dialogue:
            num_of_utterances += 1

    return num_of_utterances

# Count average tokens per utterance for dialogue
def count_avg_tokens_per_utterance(dialogues):

    num_of_utterances = count_utterances(dialogues)
    tokenized_dialogues = tokenize_data(dialogues)
    total_tokens = 0

    for dialogue in tokenized_dialogues:
        for utterance in dialogue:
            total_tokens += len(utterance)

    avg_tokens_per_utterance = total_tokens / num_of_utterances

    return avg_tokens_per_utterance


# Uses NLTK part of speech tagger to identify pronouns, counts
# the number of pronouns and then the average per utterance
def avg_pronouns_per_utterance(dialogues):
    
    tokenized_dialogues = tokenize_data(dialogues)
    pronoun_count = 0

    for dialogue in tokenized_dialogues:
        for utterance in dialogue:
            tagged_utterance = pos_tag(utterance)

            for (token, prp_tag) in tagged_utterance:
                if prp_tag == ('PRP' or 'PRP$'):
                    pronoun_count += 1

    num_of_utterances = count_utterances(dialogues)
    avg_prp = pronoun_count / num_of_utterances

    return avg_prp


# Didn't find any clear resource for agreement or negation wording.
# There is nltk.metrics.agreement, but it is not for counting agreement words
# There is also the option to try and find negation/agreement related words through wordnet, but it would also find words that are not specifially negation/agreement words
# The custom list of agreement/negation words is subject to change
# choice = 1 counts average number of agreement words
# choice = 2 does the same for negation words
def avg_agreement_negation_per_utterance(dialogues, choice):

    agreement_words = ['yes', 'ok', 'sure', 'okay', 'agreed', 'agree']
    negation_words = ['no', 'not', "don't", "can't", 'neither', ]

    if choice == 1:
        words_to_count = agreement_words
    elif choice == 2:
        words_to_count = negation_words
    else:
        print("Second argument: 1 for agreement words, 2 for negation words")
        return 0

    tokenized_dialogues = tokenize_data(dialogues)
    num_of_utterances = count_utterances(dialogues)
    num_words_to_count = 0

    for dialogue in tokenized_dialogues:
        for utterance in dialogue:
            for token in utterance:
                if token in words_to_count:
                    num_words_to_count = num_words_to_count + 1
    
    avg_agreement_negation = num_words_to_count / num_of_utterances

    return avg_agreement_negation

# Prints all stats for a given topic
def print_stats_from_excel(name_of_excel_file):

    if '.xlsx' not in name_of_excel_file:
        name_of_excel_file = name_of_excel_file + '.xlsx'

    dialogues = open_process_data(name_of_excel_file)
    vocab = vocabulary_size(dialogues)
    utterances = count_utterances(dialogues)
    tokens_per_utterance = count_avg_tokens_per_utterance(dialogues)
    avg_prp = avg_pronouns_per_utterance(dialogues)
    avg_agreement = avg_agreement_negation_per_utterance(dialogues, 1)
    avg_negation = avg_agreement_negation_per_utterance(dialogues, 2)

    print("Stats for file \"" + name_of_excel_file + "\":")
    print("Size of vocabulary: " + str(vocab))
    print("Number of utterances: " + str(utterances))
    print("Average number of tokens per utterance: " + str(tokens_per_utterance))
    print("Average number of pronouns per utterance: " + str(avg_prp))
    print("Average number of agreement words per utterance: " + str(avg_agreement))
    print("Average number of negation words per utterance: " + str(avg_negation))


# Give name of the excel data file, where the dialogue data saved with previous block is stored
print_stats_from_excel('topic9data')

Stats for file "topic9data.xlsx":
Size of vocabulary: 1567
Number of utterances: 1611
Average number of tokens per utterance: 6.848541278708876
Average number of pronouns per utterance: 0.728739913097455
Average number of agreement words per utterance: 0.032898820608317815
Average number of negation words per utterance: 0.0446927374301676


Task 2
Add task description

In [12]:
# Opens an excel data file saved in the format that task1_save_topic.py saves and
# calculates the entity tags per utterance
def avg_person_organization_entity_tags_per_utterance(excel_file_name):

    if '.xlsx' not in excel_file_name:
        excel_file_name = excel_file_name + '.xlsx'

    entity_tagger = spacy.load("en_core_web_md")
    dialogues = open_process_data(excel_file_name)
    num_entities = 0

    for dialogue in dialogues:
        for utterance in dialogue:
            entity_tagged_utterance = entity_tagger(utterance)

            for entity in entity_tagged_utterance.ents:
                if entity.label_ == ("ORG" or "PERSON"):
                    num_entities += 1

    num_utterances = count_utterances(dialogues)
    avg_ent_tag_per_utterance = num_entities / num_utterances

    return avg_ent_tag_per_utterance

# Give name of the excel data file, where the dialogue data is saved
ent_tags_avg = avg_person_organization_entity_tags_per_utterance('topic9data')
print("Average number of person/organization named-entities per utterance: " + str(ent_tags_avg))

Average number of person/organization named-entities per utterance: 0.011793916821849782


Task 3
Add task description

In [14]:
# Loads and parses dialogues from a text file dialogues_text.txt
# separating each utterance by __eou__ and returns a list of lists (one per dialogue).
def get_dialogs():

    with open("dialogues_text.txt", "r", encoding="utf-8") as file:
        dialogs = file.readlines()

    parsed_dialogs = []
    for dialog in dialogs:
        d = dialog.split("__eou__")
        d = d[:-1]
        parsed_dialogs.append(d)

    return parsed_dialogs

# Tokenizes each utterance and tags each word’s part of speech (POS).
# Queries WNAffect to get emotions for each word in the utterance based on POS tags, accumulating any detected emotions in a list.
# Returns list of emotions for utterance
def get_emotions(utterance):

    tokens = word_tokenize(utterance)
    pos_tags = pos_tag(tokens)

    emotions = []
    for i in range(len(tokens)):
        emo = wna.get_emotion(tokens[i], pos_tags[i][1])
        if emo != None:
            emotions.append(emo.name)
            Emotion.printTree(Emotion.emotions[emo.name])
            parent = emo.get_level(emo.level - 1)
            print("parent: " + parent.name)

    return emotions

# Compares the predicted emotions against expected labels from dialogues_emotion.txt.
# Calculates accuracy and precision metrics by comparing the predicted emotions (from emos.json) with the labels.
#   Consider as match if at least one emos.json label matches with tag from dialogues_emotion.txt.
# validate uses a confusion matrix approach with:
#   True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN).
# Prints accuracy and precision scores.
def validate(emotions):

    with open("dialogues_emotion.txt", "r", encoding="utf-8") as file:
        emotion_numbers = file.readlines()

    en = []
    for e in emotion_numbers:
        a = e.split(" ")
        a = a[:-1]
        en.append(a)

    emo_tags = {0: "no emotion", 1: "anger", 2: "disgust", 3: "fear", 4: "happiness", 5: "sadness", 6: "surprise"}

    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(emotions)):
        for j in range(len(emotions[i])):
            try:
                tag = emo_tags[int(en[i][j])]
                utterance_emo = emotions[i][j]
                if tag in utterance_emo:
                    tp += 1
                elif tag == "no emotion":
                    if len(utterance_emo) == 0:
                        tn += 1
                    else:
                        fn += 1
                else:
                    fp += 1
            except IndexError:
                print("index error: " + str(i) + " " + str(j))

    total = fp + fn + tp + tn

    accuracy = (tp + tn) / total
    precision = tp / (tp + fn)
    
    print("WNAffect scores: ")
    print("Total utterance count: " + str(total))
    print("Accuracy: " + str(round(accuracy ,3)))
    print("Precision: " + str(round(precision, 3)))

    return

# Calls get_dialogs to load dialogues.
# Saves emotions fro each utterance to emos.json
def save_emotions():
    dialogs = get_dialogs()

    emotions = []
    for dialog in dialogs:
        dialog_emo = []
        for utterance in dialog:
            emo = get_emotions(utterance)
            dialog_emo.append(emo)
        emotions.append(dialog_emo)

    with open("emos.json", "w") as f:
        json.dump(emotions, f, indent=4)

wna = WNAffect('wordnet-1.6/', 'wn-domains-3.2/')

# Comment here about the output and what this and validate(data) does
with open("emos.json", "r") as f:
    data = json.load(f)

validate(data)

NameError: name 'WNAffect' is not defined

Task 4
Add task description

In [15]:
# Comment here what this does

analyzer = SentimentIntensityAnalyzer()
dialogs = get_dialogs()

#print(analyzer.polarity_scores("Isn’t he the best instructor? I think he’s so hot. Wow! I really feel energized, dont’t you?"))

dialog_sentiments = []
for dialog in dialogs:
    utterances = []
    for utterance in dialog:
        vs = analyzer.polarity_scores(utterance)
        utterances.append(vs)
        print("{:-<65} {}".format(utterance, str(vs)))
    dialog_sentiments.append(utterances)

with open("sentiments.json", "w", encoding="utf-8") as file:
    json.dump(dialog_sentiments, file, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'dialogues_text.txt'

Task 5
Add task description

In [16]:
# Add comments to all functions and what main does

def save_upper_level_emotions():
    dialogs = get_dialogs()

    emotions = []
    for dialog in dialogs:
        dialog_emo = []
        for utterance in dialog:
            emo = get_emotions(utterance)
            dialog_emo.append(emo)
        emotions.append(dialog_emo)

    with open("emos_upper_level.json", "w") as f:
        json.dump(emotions, f, indent=4)

def get_emotions(utterance):

    tokens = word_tokenize(utterance)
    pos_tags = pos_tag(tokens)

    emotion_tags = []
    for i in range(len(tokens)):
        emo = wna.get_emotion(tokens[i], pos_tags[i][1])
        if emo != None:
            emotion = get_upper_level_emotion(emo)
            emotion_tags.append(emotion)
            # Emotion.printTree(Emotion.emotions[emo.name])
            # parent = emo.get_level(emo.level - 1)
            # print("parent: " + parent.name)

    return emotion_tags

def get_upper_level_emotion(emo):

    parent = emo.get_level(emo.level - 1)

    while parent.name != "negative-emotion" and parent.name != "positive-emotion" and parent.name != "positive-emotion" and parent.name != "ambiguous-emotion" and parent.name != "neutral-emotion":
        parent = emo.get_level(parent.level - 1)
    
    if parent.name == "ambiguous-emotion" or parent.name == "neutral-emotion":
        return 0
    elif parent.name == "negative-emotion":
        return -1
    else:
        return 1

def compare_and_save():

    with open("emos_upper_level.json", "r") as file:
        upper_level = json.load(file)
    
    with open("sentiments.json", "r") as file:
        sentiments = json.load(file)

    compared_index = []
    for i in range(len(upper_level)):
        dialogs = []
        for j in range(len(upper_level[i])):
            result = get_compared_result(sentiments[i][j]["compound"], upper_level[i][j])
            dialogs.append(result)
        compared_index.append(dialogs)

    save_to_excel(compared_index, upper_level, sentiments, get_dialogs())
    
def save_to_excel(compared_index, emotion_values, sentiments, dialogs):
    data = {}
    compability_index_list = []
    emotion_values_list = []
    sentiments_list = []
    utterances_list = []
    for i in range(len(compared_index)):
        compability_index_list.extend(compared_index[i])
        for j in range(len(compared_index[i])):
            if len(emotion_values[i][j]) == 0:
                emotion_values_list.append("None")
            else:
                emotion_values_list.append(emotion_values[i][j])
            sentiments_list.append(sentiments[i][j]["compound"])
            utterances_list.append(dialogs[i][j])
    data["compability index"] = compability_index_list
    data["emotion value"] = emotion_values_list
    data["sentiment"] = sentiments_list
    data["utterance"] = utterances_list

    df = pd.DataFrame(data)
    df.to_excel("task5_data.xlsx", index=False)

def get_compared_result(sentiment_value, emotion_values):
    if sentiment_value >= 0.05:
        if 1 in emotion_values:
            if 0 not in emotion_values and -1 not in emotion_values:
                return 1
            return 0.5
        else:
            return 0
    elif sentiment_value <= -0.05:
        if -1 in emotion_values:
            if 1 not in emotion_values and 0 not in emotion_values:
                return 1
            return 0.5
        else:
            return 0
    else:
        if 0 in emotion_values:
            if 1 not in emotion_values and -1 not in emotion_values:
                return 1
            return 0.5
        else:
            return 0
        
# Comment here
compare_and_save()

FileNotFoundError: [Errno 2] No such file or directory: 'dialogues_text.txt'

Task 6
Add task description

In [19]:
# Opens excel data and saves it into a multidimensional list
# Use task1_save_topic.py to save topic dialogues in this excel form
def open_process_data(name_of_data_file):
    
    data = pd.read_excel(name_of_data_file, header=None)
    rows, columns = data.shape
    dialogs = []

    for row in range(rows):
        utterances = []

        for column in range(columns):
            if (type(data.iat[row, column]) is str) & (data.iat[row, column] != "\n"):
                utterances.append(data.iat[row, column])
            
        dialogs.append(utterances)

    return dialogs

# Feature extraction function as specified in the NLTK organization book chapter 6, section 2.2
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

# Trains the classifier as specified in the NLTK organization book chapter 6, section 2.2
def train_NLTK_model():

    nltk.download('nps_chat')
    posts = nltk.corpus.nps_chat.xml_posts()[:10000]

    featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
    size = int(len(featuresets) * 0.1)
    train_set, test_set = featuresets[size:], featuresets[:size]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    print("Classifier accuracy on the NLTK NPS corpus: " + str(nltk.classify.accuracy(classifier, test_set)))

    return classifier

# Classifies data from an excel file
def classify_data(name_of_data_file):

    if '.xlsx' not in name_of_data_file:
        name_of_data_file = name_of_data_file + '.xlsx'

    data = open_process_data(name_of_data_file)
    classifier = train_NLTK_model()

    predictions = []

    for dialog in data:
        dialog_predictions = []

        for utterance in dialog:
            utterance_features = dialogue_act_features(utterance)
            prediction = classifier.classify(utterance_features)
            dialog_predictions.append(prediction)

        predictions.append(dialog_predictions)

    return predictions
    
# Saves predictions
def save_to_json(predictions, saved_file_name):

    with open(saved_file_name, "w") as file:
        json.dump(predictions, file, indent=4)

# Give name of the excel data file, where the dialogue data is saved
predictions = classify_data('topic9data')
save_to_json(predictions, 'topic9data_test')

[nltk_data] Downloading package nps_chat to /home/protago/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


Classifier accuracy on the NLTK NPS corpus: 0.667


Task 7
Add task description
Pearson correlation and p-values

In [21]:
# Comment here
def calculate_correlations(dialogue_acts_data_file, emotions_file, sentiments_file):

    with open(dialogue_acts_data_file, 'r', encoding='utf-8') as file:
        dialogue_acts_data = json.load(file)

    with open(emotions_file, 'r', encoding='utf-8') as file:
        emotions = json.load(file)

    with open(sentiments_file, 'r', encoding='utf-8') as file:
        sentiments = json.load(file)

    # The first element in the list holds a dictionary that has emotions as keys and the number of times that
    # emotion has appeared in the same utterance for that dialogue act as values
    # The second element in the list holds the compound sentiment and third keeps track of how many compound
    # sentiments were added together
    correlations = {
        "Emotion": [{}, 0, 0],
        "yAnswer": [{}, 0, 0],
        #"yAnswer" : [{}, 0, 0],
        "Continuer": [{}, 0, 0],
        "whQuestion": [{}, 0, 0],
        "System": [{}, 0, 0],
        "Accept": [{}, 0, 0],
        "Clarify": [{}, 0, 0],
        #"Clarity": [{}, 0, 0],
        "Emphasis": [{}, 0, 0],
        "nAnswer": [{}, 0, 0],
        "Greet": [{}, 0, 0],
        "Statement": [{}, 0, 0],
        "Reject": [{}, 0, 0],
        "Bye": [{}, 0, 0],
        "Other" : [{}, 0, 0],
        #"Others" : [{}, 0, 0],
        "ynQuestion" : [{}, 0, 0],
    }

    # Iterate over each utterance in each dialogue
    for dialogue in range(len(dialogue_acts_data)):
        for utterance in range(len(dialogue_acts_data[dialogue])):

            
            dialogue_act = dialogue_acts_data[dialogue][utterance]
            sentiment = sentiments[dialogue][utterance]

            # If emotions list is empty, then emotions[dialogue][utterance][0] doens't exist,
            # but if it has an emotion, then emotions[dialogue][utterance] is a list and not a string
            if len(emotions[dialogue][utterance]) != 0:
                emotion = emotions[dialogue][utterance][0]
            else:
                emotion = 'NaN'

            # Adds emotion or increments emotion value in the correlations dict lists first dictionary
            # If emotion is not found before for this dialogue act, it is added to the dict as a key, with a value of one.
            # Otherwise the value for that key is incremented by one
            # If emotion is nan, do nothing
            if emotion == 'NaN':
                a = 1
            elif emotion in correlations[dialogue_act][0]:
                correlations[dialogue_act][0][emotion] += 1
            else:
                correlations[dialogue_act][0][emotion] = 1

            # Adds compound sentiment to the correlations dict lists second element
            correlations[dialogue_act][1] += sentiment["compound"]
            correlations[dialogue_act][2] += 1

    # This list stores lists, which have two elements. First is a dialogue act, second is the emotion that dialogue act has
    # the highest correlation with, i.e. they appear the most together
    highest_emotion_correlations = []

    for dialogue_act in correlations:
        
        # Get average compound sentiment
        correlations[dialogue_act][1] = correlations[dialogue_act][1] / correlations[dialogue_act][2]
        print("Compound sentiment for " + str(dialogue_act) + " is: " + str(correlations[dialogue_act][1]))

        # For each dialogue act, find emotion that appears the most with it
        if len(correlations[dialogue_act][0]) != 0:
            highest_emotion_correlations.append([dialogue_act, max(correlations[dialogue_act][0], key=correlations[dialogue_act][0].get)])

    return correlations, highest_emotion_correlations

# Comment here
correlations, highest_emotion_correlations = calculate_correlations('all_dialogues_predictions.json', 'emos.json', 'sentiments.json')
print(highest_emotion_correlations)

Compound sentiment for Emotion is: -0.19733333333333333
Compound sentiment for yAnswer is: 0.4125599510318254
Compound sentiment for Continuer is: 0.14945422885572143
Compound sentiment for whQuestion is: 0.10348610379266035
Compound sentiment for System is: 0.06702743764172334
Compound sentiment for Accept is: 0.36048351449275456
Compound sentiment for Clarify is: 0.24550313324404716
Compound sentiment for Emphasis is: 0.281300698757766
Compound sentiment for nAnswer is: 0.0759587800369691
Compound sentiment for Greet is: 0.019240740740740742
Compound sentiment for Statement is: 0.19866943847343674
Compound sentiment for Reject is: 0.12184455385363421
Compound sentiment for Bye is: 0.17189492385786803
Compound sentiment for Other is: 0.2135223988439305
Compound sentiment for ynQuestion is: 0.1458641363683198
[['yAnswer', 'stupefaction'], ['Continuer', 'stupefaction'], ['whQuestion', 'benevolence'], ['Accept', 'benevolence'], ['Clarify', 'stupefaction'], ['Emphasis', 'stupefaction'], [

Task 8
Add task description

In [ ]:
# Implementation here